In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split

falseGroup = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/no/"
trueGroup  = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/yes/"

# Get the list of file paths
false_images = [os.path.join(falseGroup, fname) for fname in os.listdir(falseGroup)]
true_images = [os.path.join(trueGroup, fname) for fname in os.listdir(trueGroup)]

# Create labels
false_labels = [0] * len(false_images)  # Label '0' for no tumor
true_labels = [1] * len(true_images)    # Label '1' for tumor present

# Combine the lists of file paths and labels
file_paths = false_images + true_images
labels = false_labels + true_labels

# Split into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# Define a function to load and preprocess images
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # Assuming JPEG format
    image = tf.image.resize(image, [224, 224])  # Resize to the input size of the model
    image = image / 255.0  # Normalize to [0,1]
    return image, label

# Create training and validation datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(load_image).batch(32).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.map(load_image).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Example: Create and train a ResNet50 model
def create_resnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create the ResNet50 model
resnet_model = create_resnet_model()

# Train the model
history = resnet_model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Evaluate on the validation dataset
val_loss, val_accuracy = resnet_model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# You can repeat the same process for other models (EfficientNet, VGG16, etc.)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time
import tensorflow as tf
import pandas as pd

# Model creation functions
def create_resnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_vgg_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_mobilenet_model(input_shape=(224, 224, 3), num_classes=1):
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_custom_cnn(input_shape=(224, 224, 3), num_classes=1):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model evaluation function
def evaluate_model(model, train_dataset, val_dataset):
    # Start the training timer
    start_time = time.time()
    
    # Train the model
    model.fit(train_dataset, epochs=10, validation_data=val_dataset)
    
    # Stop the timer for training time
    train_time = time.time() - start_time
    
    # Predict on validation data
    val_images, val_labels = next(iter(val_dataset.unbatch().batch(len(val_dataset))))
    y_pred = (model.predict(val_images) > 0.5).astype(int)
    y_true = val_labels.numpy()

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    
    # Calculate inference time
    inference_start = time.time()
    _ = model.predict(val_images)
    inference_time = (time.time() - inference_start) / len(val_images)
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "auc": auc,
        "train_time": train_time,
        "inference_time": inference_time
    }

# Prepare to run evaluation on all models
model_functions = {
    'ResNet50': create_resnet_model,
    'VGG16': create_vgg_model,
    'EfficientNetB0': create_efficientnet_model,
    'MobileNetV2': create_mobilenet_model,
    'Custom CNN': create_custom_cnn
}

results = {}

# Iterate through the models and evaluate each one
for model_name, model_function in model_functions.items():
    print(f"Training and evaluating {model_name}...")
    model = model_function()  # Create the model
    results[model_name] = evaluate_model(model, train_dataset, val_dataset)

# Display the results
df_results = pd.DataFrame(results).T  # Transpose for easier viewing
print(df_results)  # Display the DataFrame in console

| Model            | Accuracy | Precision | Recall | F1-Score | AUC  | Train Time  | Inference Time |
|------------------|----------|-----------|--------|----------|------|-------------|----------------|
| MobileNetV2      | 1.0      | 1.0       | 1.0    | 1.000000 | 1.0  | 63.973554   | 0.061566       |
| Custom CNN       | 1.0      | 1.0       | 1.0    | 1.000000 | 1.0  | 139.771326  | 0.055201       |
| ResNet50         | 1.0      | 1.0       | 1.0    | 1.000000 | 1.0  | 204.121166  | 0.115547       |
| VGG16            | 1.0      | 1.0       | 1.0    | 1.000000 | 1.0  | 762.232632  | 0.268409       |
| EfficientNetB0   | 0.5      | 0.5       | 1.0    | 0.666667 | 0.5  | 119.355642  | 0.075382       |


